In [1]:
import os
import requests
import pandas as pd
import time
from datetime import datetime


In [10]:
def get_products(category="103004", top_n=50):
    url = f"https://api.musinsa.com/api2/dp/v1/plp/goods?gf=A&category={category}&size={top_n}&caller=CATEGORY&page=1"
    res = requests.get(url, headers={"user-agent": "Mozilla/5.0"})
    res.raise_for_status()
    items = res.json().get("data", {}).get("list", [])

    return pd.DataFrame([{
        "product_id": item.get("goodsNo"),
        "brandName": item.get("brandName"),
        "goodsName": item.get("goodsName"),
        "price": item.get("price"),
        "reviewCount": item.get("reviewCount"),
        "reviewScore": item.get("reviewScore"),
        "thumbnail": item.get("thumbnail"),
        "goodsLinkUrl": item.get("goodsLinkUrl"),
    } for item in items])


def get_reviews(goods_no, max_reviews=50, page_size=20):
    """상품별 리뷰 가져오기"""
    reviews, collected = [], 0
    base_url = "https://goods.musinsa.com/api2/review/v1/view/list"

    for page in range(1, 21):
        if collected >= max_reviews:
            break
        url = f"{base_url}?page={page}&pageSize={page_size}&goodsNo={goods_no}"
        res = requests.get(url, headers={"user-agent": "Mozilla/5.0"})
        if res.status_code != 200:
            break

        items = res.json().get("data", {}).get("list", [])
        if not items:
            break

        for r in items:
            if collected >= max_reviews:
                break
            raw_date = r.get("createDate")
            dt = datetime.fromisoformat(raw_date.replace("Z", "+00:00")) if raw_date else None
            reviews.append({
                "product_id": goods_no,
                "createDate": dt.strftime("%Y-%m-%d") if dt else None,
                "review_no": r.get("no") or r.get("reviewNo") or r.get("reviewId"),
                "userNickName": r.get("userProfileInfo", {}).get("userNickName"),
                "content": r.get("content"),
                "grade": r.get("grade"),
            })
            collected += 1
        time.sleep(0.2)

    return reviews

In [ ]:
target_category = "103004"
num_products = 60
max_reviews = 300

products_df = get_products(category=target_category, top_n=num_products)
all_reviews = []

for _, row in products_df.iterrows():
    goods_no = row["product_id"]
    product_reviews = get_reviews(goods_no, max_reviews=max_reviews)
    all_reviews.extend(product_reviews)
    print(f"상품 {goods_no}의 리뷰 {len(product_reviews)}개 수집")

reviews_df = pd.DataFrame(all_reviews)

if not reviews_df.empty:
    reviews_df["createDate"] = pd.to_datetime(reviews_df["createDate"])
    reviews_df = (
        reviews_df.sort_values(by="createDate", ascending=False)
                  .drop_duplicates(subset=["product_id", "userNickName"], keep="first")
    )

print(f"\n총 상품 개수: {len(products_df)}개")
print(f"총 리뷰 개수(중복 제거 후): {len(reviews_df)}개")

products_df.to_csv("data/products.csv", index=False, encoding="utf-8-sig")
print("'data/products.csv' 저장 완료")

reviews_df.to_csv("data/reviews.csv", index=False, encoding="utf-8-sig")
print("'data/reviews.csv' 저장 완료")

상품 1473136의 리뷰 300개 수집
상품 4903311의 리뷰 300개 수집
상품 4798797의 리뷰 126개 수집
상품 4328804의 리뷰 300개 수집
상품 4232329의 리뷰 300개 수집
상품 4751371의 리뷰 153개 수집
상품 4201458의 리뷰 300개 수집
상품 4709672의 리뷰 0개 수집
상품 4963524의 리뷰 0개 수집
상품 5057337의 리뷰 0개 수집
상품 4264377의 리뷰 231개 수집
상품 4904245의 리뷰 4개 수집
상품 4729861의 리뷰 53개 수집
상품 4748066의 리뷰 60개 수집
상품 4923314의 리뷰 61개 수집
상품 4497489의 리뷰 38개 수집
상품 4487816의 리뷰 146개 수집
상품 4557500의 리뷰 300개 수집
상품 1853613의 리뷰 300개 수집
상품 4708803의 리뷰 115개 수집
상품 5186758의 리뷰 0개 수집
상품 5193964의 리뷰 0개 수집
상품 4489820의 리뷰 288개 수집
상품 5110955의 리뷰 241개 수집
상품 3794771의 리뷰 300개 수집
상품 1115031의 리뷰 300개 수집
상품 3571742의 리뷰 89개 수집
상품 4341574의 리뷰 70개 수집
상품 5163383의 리뷰 15개 수집
상품 4763425의 리뷰 0개 수집
상품 2545799의 리뷰 300개 수집
상품 4746416의 리뷰 0개 수집
상품 3549871의 리뷰 300개 수집
상품 5050693의 리뷰 0개 수집
상품 5269347의 리뷰 0개 수집
상품 5075839의 리뷰 0개 수집
상품 2815900의 리뷰 300개 수집
상품 3863540의 리뷰 300개 수집
상품 3157671의 리뷰 300개 수집
상품 2921406의 리뷰 300개 수집
상품 3442134의 리뷰 162개 수집
상품 4883765의 리뷰 0개 수집
상품 4706679의 리뷰 73개 수집
상품 5048532의 리뷰 300개 수집
상품 3886488의 리뷰 300개 